In [16]:
import os
import pandas as pd
import numpy as np
import sqlite3

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=6, use_memory_fs=False)
os.environ['JOBLIB_TEMP_FOLDER'] = '/tmp' 

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [17]:
mydata = './original.snpeff_parsed.tsv'
mydb = '../../TestData/ycudb/ycudb.db'

# Load my data
df = pd.read_csv(mydata, sep='\t', header=0, index_col=False, dtype=str)
df['DNA_ID'] = df['sample'].str.split('_').str[1]

# Connect to db
conn = sqlite3.connect(mydb)
c = conn.cursor()

In [18]:
def __fetch_state_from_mailed(query_dna_id) -> str:
    sql = f"SELECT State FROM mailed_samples WHERE DNA_ID = '{query_dna_id}'"
    try:
        return c.execute(sql).fetchone()[0]
    except TypeError:
        return 'None'
    
def __fetch_state_from_new(query_dna_id) -> str:
    sql = f"SELECT State FROM new_samples WHERE DNA_ID = '{query_dna_id}'"
    try:
        return c.execute(sql).fetchone()[0]
    except TypeError:
        return 'None'

def annotate_state(row):
    query_dna_id = row['DNA_ID']
    if __fetch_state_from_mailed(query_dna_id) != 'None':
        return __fetch_state_from_mailed(query_dna_id)
    else:
        if __fetch_state_from_new(query_dna_id) != 'None':
            return __fetch_state_from_new(query_dna_id)
        else:
            return 'Undetermined'


# Fetch disease info
def __fetch_disease_from_mailed(query_dna_id) -> str:
    sql = f"SELECT 診断名 FROM mailed_samples WHERE DNA_ID = '{query_dna_id}'"
    try:
        return c.execute(sql).fetchone()[0]
    except TypeError:
        return 'None'
    
def __fetch_disease_from_new(query_dna_id) -> str:
    sql = f"SELECT 病名 FROM new_samples WHERE DNA_ID = '{query_dna_id}'"
    try:
        return c.execute(sql).fetchone()[0]
    except TypeError:
        return 'None'
    
def __fetch_disease_from_old(query_dna_id) -> str:
    sql = f"SELECT 病名 FROM old_samples WHERE DNA_ID = '{query_dna_id}'"
    try:
        return c.execute(sql).fetchone()[0]
    except TypeError:
        return 'None'
    
def annotate_disease(row):
    query_dna_id = row['DNA_ID']
    result = []
    if __fetch_disease_from_mailed(query_dna_id) != 'None':
        result.append(__fetch_disease_from_mailed(query_dna_id))
    else:
        pass
    if __fetch_disease_from_new(query_dna_id) != 'None':
        result.append(__fetch_disease_from_new(query_dna_id))
    else:
        pass
    if __fetch_disease_from_old(query_dna_id) != 'None':
        result.append(__fetch_disease_from_old(query_dna_id))
    else:
        pass

    return ','.join(result)


df['State'] = df.parallel_apply(lambda row: annotate_state(row), axis=1)
df['Disease'] = df.parallel_apply(lambda row: annotate_disease(row), axis=1)

In [24]:
outfilename = './original.snpeff.state.disease.tsv'

In [25]:
df.to_csv(outfilename, sep='\t', index=False)